# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import pandas as pd
import os
import re
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, make_scorer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### Implement the StartingVerbExtractor class

There is something wrong with the code below when included in the ML Pipeline.

In [4]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def ML_pipeline():
    
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            #Commented due to some 'unknown' problem 
            #('starting_verb', StartingVerbExtractor())
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
            
            #('tfidf_vect', TfidfVectorizer(tokenizer=tokenize))
            
        ])),
        
#         ('clf', MultiOutputClassifier(AdaBoostClassifier()))
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = ML_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = model.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)
overall_accuracy = (y_pred == y_test).mean().mean()
print('Overall accuracy = {0:.2f}%\n'.format(overall_accuracy*100))

for column in y_test.columns:
    print('CATEGORY: {}'.format(column))
    print(classification_report(y_test[column],y_pred_df[column]))

Overall accuracy = 94.79%

CATEGORY: related
             precision    recall  f1-score   support

          0       0.56      0.24      0.34      1513
          1       0.80      0.94      0.86      4999
          2       0.24      0.10      0.14        42

avg / total       0.74      0.77      0.74      6554

CATEGORY: request
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      5408
          1       0.77      0.56      0.65      1146

avg / total       0.89      0.89      0.89      6554

CATEGORY: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

CATEGORY: aid_related
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3822
          1       0.76      0.62      0.68      2732

avg / total       0.76      0.76      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
model2 = ML_pipeline()

parameters = {
#     'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#     'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#     'features__text_pipeline__tfidf__use_idf': (True, False),
#     'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__learning_rate': [0.5, 1.0]
}

cv = GridSearchCV(model2, param_grid=parameters)

cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred2 = cv.predict(X_test)
y_pred2_df = pd.DataFrame(y_pred2, columns = y_test.columns)
overall_accuracy = (y_pred2 == y_test).mean().mean()
print('Overall accuracy = {0:.2f}%\n'.format(overall_accuracy*100))
print('Best Parameters: {}\n'.format(cv.best_params_))

for column in y_test.columns:
    print('CATEGORY: {}'.format(column))
    print(classification_report(y_test[column],y_pred2_df[column]))

Overall accuracy = 94.79%

Best Parameters: {'clf__estimator__learning_rate': 1.0}

CATEGORY: related
             precision    recall  f1-score   support

          0       0.56      0.24      0.34      1513
          1       0.80      0.94      0.86      4999
          2       0.24      0.10      0.14        42

avg / total       0.74      0.77      0.74      6554

CATEGORY: request
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      5408
          1       0.77      0.56      0.65      1146

avg / total       0.89      0.89      0.89      6554

CATEGORY: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

CATEGORY: aid_related
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3822
          1       0.76      0.62     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
#new model with random forest classifier
rf_model = ML_pipeline()
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
y_pred_rf_df = pd.DataFrame(y_pred_rf, columns = y_test.columns)
overall_accuracy = (y_pred_rf == y_test).mean().mean()
print('Overall accuracy = {0:.2f}%\n'.format(overall_accuracy*100))

Overall accuracy = 94.79%



#### Overall accuracy is the same using random forest.

### 9. Export your model as a pickle file

In [15]:
# save the model to disk
saved_model = pickle.dumps(model)

# Load the pickled model 
model_from_pickle = pickle.loads(saved_model) 
  
# Use the loaded pickled model to make predictions 
model_from_pickle.predict(X_test) 

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.